In [1]:
! pip install datasets
! pip install transformers
! pip install cleantext
! pip uninstall -y transformers accelerate
! pip install transformers accelerate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 37.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 21.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 8.6 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 39.5 MB/s eta 0:00

In [2]:
import re
import os
import sys
import cleantext
import numpy as np
import pandas as pd
from cleantext import clean
from datasets import load_metric
from transformers import Trainer
from datasets import load_dataset
from transformers import  pipeline
from transformers import AutoTokenizer
from transformers import TrainingArguments
from transformers import AutoModelForSequenceClassification
from pandas.core.arrays.integer import Int64Dtype
from transformers import RobertaTokenizer, RobertaForSequenceClassification

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

def tokenize_data(example):
    max_length = 512
    tok = tokenizer(example['text'],
                    truncation=True,
                    max_length=max_length,
                    return_tensors='pt',
                    padding='max_length')

    return tok
def remove_emoji(string):
  emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002500-\U00002BEF"  # chinese char
                               u"\U00002702-\U000027B0"
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               u"\U0001f926-\U0001f937"
                               u"\U00010000-\U0010ffff"
                               u"\u2640-\u2642"
                               u"\u2600-\u2B55"
                               u"\u200d"
                               u"\u23cf"
                               u"\u23e9"
                               u"\u231a"
                               u"\ufe0f"  # dingbats
                               u"\u3030"
                               "]+", flags=re.UNICODE)
  return emoji_pattern.sub(r'', string)

In [5]:
df = pd.read_pickle('/content/drive/MyDrive/Colab Notebooks/fine_tunnig/df_text_clean.pkl')
df = df[['title','label']]
df = df.dropna()


df = df.sample(frac = 1)
df = df.reset_index(drop = True)
df['label'] = df['label'].astype('int')
df.rename(columns={'label': 'labels',
                   "title": "text"},
          inplace=True)

df = df[['text','labels']]


df = df.reset_index(drop = True)
df_train = df.sample(frac = 0.75)
df_test = df.drop(df_train.index)
df_train['labels'] = df_train['labels'].astype('int')
df_test['labels'] = df_test['labels'].astype('int')
df_train = df_train.reset_index(drop=True)
df_test = df_test.reset_index(drop = True)
df_train.to_csv('train_data.csv',index = False)
df_test.to_csv('test_data.csv', index = False)


dataset = load_dataset('csv', data_files={'train': 'train_data.csv',
                                          'test': 'test_data.csv'},
                       encoding = "ISO-8859-1")

tokenizer = RobertaTokenizer.from_pretrained("cardiffnlp/twitter-roberta-base-sentiment")
model = RobertaForSequenceClassification.from_pretrained("cardiffnlp/twitter-roberta-base-sentiment",
                                                         num_labels=4,
                                                         ignore_mismatched_sizes=True)

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-21f759f1312657cf/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment and are newly initialized because the shapes did not match:
- classifier.out_proj.weight: found shape torch.Size([3, 768]) in the checkpoint and torch.Size([4, 768]) in the model instantiated
- classifier.out_proj.bias: found shape torch.Size([3]) in the checkpoint and torch.Size([4]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
# tokenizer = AutoTokenizer.from_pretrained('/content/drive/MyDrive/models_sentiment_emotion_bru/models/emotion_model')
dataset = dataset.map(tokenize_data, batched=True)
remove_columns = ['text']
dataset = dataset.map(remove_columns=remove_columns)

Map:   0%|          | 0/71966 [00:00<?, ? examples/s]

Map:   0%|          | 0/23989 [00:00<?, ? examples/s]

Map:   0%|          | 0/71966 [00:00<?, ? examples/s]

Map:   0%|          | 0/23989 [00:00<?, ? examples/s]

In [7]:
training_args = TrainingArguments("test_trainer",
                                  num_train_epochs=3)
# model = AutoModelForSequenceClassification.from_pretrained("/content/drive/MyDrive/models_sentiment_emotion_bru/models/emotion_model", num_labels=5, ignore_mismatched_sizes=True)

a = int(len(df_train)*0.75)
train_dataset = dataset['train'].shuffle(seed=10).select(range(0,a))
eval_dataset = dataset['train'].shuffle(seed=10).select(range(a,len(df_train)))
trainer = Trainer(model=model,
                  args=training_args,
                  train_dataset=train_dataset,
                  eval_dataset=eval_dataset)
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss
500,0.632400
1000,0.598100
1500,0.562700
2000,0.545500
2500,0.564100
3000,0.584500
3500,0.578900
4000,0.602000
4500,0.641400
5000,0.597300


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /usr/local/lib/python3.10/dist-packages/torch/serialization.py:441 in save                       │
│                                                                                                  │
│    438 │                                                                                         │
│    439 │   if _use_new_zipfile_serialization:                                                    │
│    440 │   │   with _open_zipfile_writer(f) as opened_zipfile:                                   │
│ ❱  441 │   │   │   _save(obj, opened_zipfile, pickle_module, pickle_protocol)                    │
│    442 │   │   │   return                                                                        │
│    443 │   else:                                                                                 │
│    444 │   │   with _open_file_like(f, 'wb') as opened_file:                                     │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/torch/serialization.py:668 in _save                      │
│                                                                                                  │
│    665 │   │   │   storage = storage.cpu()                                                       │
│    666 │   │   # Now that it is on the CPU we can directly copy it into the zip file             │
│    667 │   │   num_bytes = storage.nbytes()                                                      │
│ ❱  668 │   │   zip_file.write_record(name, storage.data_ptr(), num_bytes)                        │
│    669                                                                                           │
│    670                                                                                           │
│    671 def load(                                                                                 │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
RuntimeError: [enforce fail at inline_container.cc:471] . PytorchStreamWriter failed writing file data/1: file 
write failed

During handling of the above exception, another exception occurred:

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 12>:12                                                                            │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/trainer.py:1664 in train                    │
│                                                                                                  │
│   1661 │   │   inner_training_loop = find_executable_batch_size(                                 │
│   1662 │   │   │   self._inner_training_loop, self._train_batch_size, args.auto_find_batch_size  │
│   1663 │   │   )                                                                                 │
│ ❱ 1664 │   │   return inner_training_loop(                                                       │
│   1665 │   │   │   args=args,                                                                    │
│   1666 │   │   │   resume_from_checkpoint=resume_from_checkpoint,                                │
│   1667 │   │   │   trial=trial,                                                                  │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/trainer.py:2019 in _inner_training_loop     │
│                                                                                                  │
│   2016 │   │   │   │   │   self.state.epoch = epoch + (step + 1 + steps_skipped) / steps_in_epo  │
│   2017 │   │   │   │   │   self.control = self.callback_handler.on

In [ ]:
metric = load_metric("precision")
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,
    
)
trainer.evaluate()

In [ ]:
save_directory = '/content/drive/MyDrive/Colab Notebooks/fine_tunnig'
tokenizer.save_pretrained(save_directory)
model.save_pretrained(save_directory)

In [ ]:
 
model_name = "/content/drive/MyDrive/Colab Notebooks/fine_tunnig"
model_text = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer_sentiment = AutoTokenizer.from_pretrained(model_name)
classifier_sentiment = pipeline("text-classification", model=model_text, tokenizer=tokenizer_sentiment)


In [ ]:
text="asdnkflasñdf sdkmgsdf g"
result = classifier_sentiment(text)
result 

In [ ]:
FN=0
TN=0
TP=0
FP=0
for i in range (0,len( df_test)):
  text = df_test['text'][i]
  result = classifier_sentiment(text)
  if result[0]['label'] == 'LABEL_1' and df_test['labels'][i]== 1:
    TP +=1
  if result[0]['label'] == 'LABEL_1' and df_test['labels'][i]== 0:
    FP +=1
  if result[0]['label'] == 'LABEL_0' and df_test['labels'][i]== 0:
    TN +=1
  if result[0]['label'] == 'LABEL_0' and df_test['labels'][i]== 1:
    FN +=1
acc = (TP+TN)/(TP+TN+FP+FN)
precision = TP /(TP + FP)
recall = TP /(TP +FN)

In [ ]:
acc = (TP+TN)/(TP+TN+FP+FN)
precision = TP /(TP + FP)
recall = TP /(TP + FN)
print("accuracy: ",acc)
print("precision:", precision)
print("recall:", recall)